<a href="https://colab.research.google.com/github/gluo88/misc/blob/main/notebook/finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Example 1:total returns and trailing total returns - multiple tickers

In [16]:
"""
### CODE start (for AI tool)
Retrieve multiple tickers for total returns and trailing total returns from yahoo
History:
Dec 21, 2023, version 12

Author: Gang Luo
"""
result_marker='###'
script_version = 12
print(f"The result of version-{script_version} script  start:\n {result_marker}")
#!pip install yfinance
import yfinance as yf
import pandas as pd

# Set the stock tickers list
#tickers_list = ["spy", "vfv.to", "xiu.to", "xic.to", "xfn.to", "ry.to", "enb.to"]
tickers_list = ["xiu.to", "xfn.to", "ry.to", "td.to", "na.to", "cwb.to", "slf.to", "gwo.to"]
#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display

# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    ''' Get annual total return data.   Explanation the following statement
    1.  'Close'  in .Ticker(ticker).history is the same as 'Adj Close' in .download(ticker, ...)
        print(df.columns.tolist()) - Display all column names of DataFrame
        yf.Ticker(ticker).history: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
        yf.download(ticker, ...): ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    2.  ["Close"]: Selects the "Close" column from the historical stock data.
        .resample('Y'): Resamples the time series data annually ('Y').
        .ffill(): Forward fills missing values, ensuring that each annual period has a value.
        .pct_change(): Calculates the percentage change between the current and previous values, representing the annual returns.
        .dropna(): Removes any rows with missing values (which may occur after calculating percentage changes).
    '''
    annual_returns = stock.history(period="max")["Close"].resample('Y').ffill().pct_change().dropna()
    annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
    annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
    return annual_returns_df
# Create an empty DataFrame to store all tickers' total returns
all_tickers_returns_df = pd.DataFrame()

# Loop through each ticker in the list
for ticker in tickers_list:
    ticker_returns_df = get_annual_returns_df(ticker)
    if not ticker_returns_df.empty:
        if all_tickers_returns_df.empty:
            all_tickers_returns_df = ticker_returns_df
        else:
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df], axis=1)  # Concatenate DataFrames

# Display the results if the DataFrame is not empty
print("\nAnnual Total Return (%) History:")
if not all_tickers_returns_df.empty:
    # keep the date portion of index only - stripping time portion
    #all_tickers_returns_df.index=all_tickers_returns_df.index.date

    # Format the DataFrame to display percentage
    all_tickers_returns_df_copy = all_tickers_returns_df * 100
    all_tickers_returns_df_copy=all_tickers_returns_df_copy.round(2)
    all_tickers_returns_df_copy.index=all_tickers_returns_df_copy.index.date
    print(all_tickers_returns_df_copy)
else:
    print("No data available for the specified tickers.")

#-------------------------------------------------------------------
# step 2: Remove the row of the current year for the later steps
# Define a list of years to calculate the trailing returns, cumulative returns, and so on
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30]
current_year=all_tickers_returns_df.index.year.max()
all_tickers_returns_df['Year']=all_tickers_returns_df.index.year
all_tickers_returns_df=all_tickers_returns_df[all_tickers_returns_df['Year'] < current_year]


#-------------------------------------------------------------------
# step 3: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)

    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)

        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return "N/A"

# Create an empty DataFrame with years_list as the index for trailing total returns
all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in all_tickers_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = all_tickers_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = "N/A"
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

# Loop through each ticker in the list
for ticker in tickers_list:
    trailing_returns = get_trailing_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values

# Display the trailing returns DataFrame
print("\nAnnualized Trailing Returns as end of year", current_year-1, ":")
print("years")
print(all_tickers_trailing_returns_df)

# print an indicator to mark the end of execution result
print(f"The result of version-{script_version} script  ends here.\n {result_marker}")
### CODE end (for AI tool)


The result of version-12 script  start:
 ###

Annual Total Return (%) History:
            xiu.to  xfn.to  ry.to  td.to   na.to  cwb.to  slf.to  gwo.to
1996-12-31     NaN     NaN  59.20  52.30   29.73   38.08     NaN     NaN
1997-12-31     NaN     NaN  61.23  56.87   75.97   62.90     NaN     NaN
1998-12-31     NaN     NaN   3.58   2.50    7.92   10.75     NaN     NaN
1999-12-31     NaN     NaN -14.82  46.56  -22.65  -21.16     NaN     NaN
2000-12-31    8.02     NaN  65.34  14.84   43.78   42.47     NaN     NaN
2001-12-31  -14.24     NaN   4.90  -2.81   11.65    9.86  -13.63     NaN
2002-12-31  -14.02   -3.06  14.85 -14.56    8.75   -5.13  -19.87     NaN
2003-12-31   24.82   26.14   9.13  31.37   35.63   50.03   23.78   25.64
2004-12-31   13.28   18.57   7.38  18.83   18.76   37.62   27.27   20.62
2005-12-31   25.47   22.90  45.87  26.19   25.57   36.52   19.14   18.24
2006-12-31   19.12   17.79  26.02  16.55   12.71   49.19    8.15   13.56
2007-12-31   10.83   -1.44  -5.57   2.66  -17

# Example 1_2 Example 1:total returns and trailing total returns - single ticker

In [ ]:
Would you please to write a detailed prompt for generating the above code that can be used to re-generate better python code for efficiency, easy understanding,   good commenting

--------------  version has been revised by myself ----
Prompt for Generating Python Code:

Objective:
The objective of this Python script is to utilize the Yahoo Finance API (yfinance) for fetching yearly total returns corresponding to a specified stock ticker. Additionally, the script aims to calculate and present the annualized trailing total return for various predefined time periods.

Steps:

Step 1: Library Imports:
Import essential libraries: yfinance as yf and pandas as pd.

Step 2: Fetch Yearly Total Returns:
Develop a function using the yfinance library (‘import yfinance as yf’, ‘ yf.Ticker(ticker).history()’)   to retrieve yearly total returns.
Apply the function to obtain historical data and compute annual returns.
Display the obtained annual total return history.

Step 3: Calculate Annualized Trailing Total Return:

Define a function to compute the annualized trailing total return based on specified time periods.
Utilize the annual total return history obtained in Step 2 for this computation.
Implement the calculation method: Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1.
Leverage Pandas for data manipulation as required.
Add the calculation method and its reference (https://www.investopedia.com/terms/a/annualized-total-return.asp) as a comment.
Display the annualized trailing total return for different time periods.

The expected execution result: ###

Annual Total Return (%) History:
                           Total Return
Date
2002-12-31 00:00:00-05:00     -0.030648
2003-12-31 00:00:00-05:00      0.261358
2004-12-31 00:00:00-05:00      0.185731
2005-12-31 00:00:00-05:00      0.229000
2006-12-31 00:00:00-05:00      0.177913
2007-12-31 00:00:00-05:00     -0.014422
2008-12-31 00:00:00-05:00     -0.359793
2009-12-31 00:00:00-05:00      0.445820
2010-12-31 00:00:00-05:00      0.079038
2011-12-31 00:00:00-05:00     -0.044104
2012-12-31 00:00:00-05:00      0.164368
2013-12-31 00:00:00-05:00      0.259991
2014-12-31 00:00:00-05:00      0.118761
2015-12-31 00:00:00-05:00     -0.037058
2016-12-31 00:00:00-05:00      0.233812
2017-12-31 00:00:00-05:00      0.125416
2018-12-31 00:00:00-05:00     -0.097634
2019-12-31 00:00:00-05:00      0.206600
2020-12-31 00:00:00-05:00      0.009886
2021-12-31 00:00:00-05:00      0.355708
2022-12-31 00:00:00-05:00     -0.099170
2023-12-31 00:00:00-05:00      0.103418
Trailing Returns as : 2022-12-31
1-Year -9.92%
2-Year 10.51%
3-Year 7.24%
5-Year 6.07%
10-Year 9.73%
15-Year 7.14%
20-Year 9.38%
###



In [ ]:
### CODE started here:
#!pip install yfinance
import yfinance as yf
import pandas as pd

print("The execution result started here: ###")

#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display

# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    # Get annual total return data
    annual_returns = stock.history(period="max")["Close"].resample('Y').ffill().pct_change().dropna()
    annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
    annual_returns_df.rename(columns={'Close': 'Total Return'}, inplace=True)
    return annual_returns_df

# Set the stock ticker
ticker = "xfn.to"
# Fetch data and extract required information
data = get_annual_returns_df(ticker)
print("\nAnnual Total Return (%) History:")
# print(data.to_string(index=True)) # Add index=True
print(data)

#-------------------------------------------------------------------
# step 2: calculate the annualized trailing total return from the data generated in step 1 & display

# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(data, years):
    # Get the total return values for the last n years
    trailing_data = data['Total Return'].tail(years)

    # Convert the percentage strings to numeric values
    #trailing_data = trailing_data.str.replace('%', '').astype(float)    # not working
    trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)

    """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
        Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
        Where r1, r2, ..., rn are the total return values for each year                    """
    #annualized_return = (trailing_data / 100 + 1).prod() ** (1 / years) - 1  # is incorrect
    annualized_return = (trailing_data + 1).prod() ** (1 / years) - 1

    # Format the result as a percentage with two decimal places
    annualized_return = "{:.2f}%".format(annualized_return * 100)

    return annualized_return

TRAIL_RETURN_AS_YEAR=2022
data.drop(data.index[-1], inplace=True)

# Assuming data is your DataFrame
data['Year'] = data.index.year
data['Date'] = data.index

# If you want to keep only the date portion without the time
data['Date'] = data['Date'].dt.date
# Remove rows where 'Year' is newer than TRAIL_RETURN_AS_YEAR
data = data[data['Year'] <= TRAIL_RETURN_AS_YEAR]
# Find the newest date and extract the date portion
newest_date = data['Date'].max()
#Print the newest date
print("Trailing Returns as :", newest_date)
#print(data['Adj Close'].tail(1))

# Define a list of years to calculate the trailing returns
years_list = [1, 2, 3, 5, 10, 15, 20]

# Loop through the list and print the trailing returns for each year
for years in years_list:
    trailing_return = get_trailing_return(data, years)
    print(f"{years}-Year {trailing_return}")

# print an indicator to mark the end of execution result
print("###")

The execution result started here: ###

Annual Total Return (%) History:
                           Total Return
Date                                   
2002-12-31 00:00:00-05:00     -0.030649
2003-12-31 00:00:00-05:00      0.261359
2004-12-31 00:00:00-05:00      0.185730
2005-12-31 00:00:00-05:00      0.229000
2006-12-31 00:00:00-05:00      0.177914
2007-12-31 00:00:00-05:00     -0.014424
2008-12-31 00:00:00-05:00     -0.359792
2009-12-31 00:00:00-05:00      0.445820
2010-12-31 00:00:00-05:00      0.079037
2011-12-31 00:00:00-05:00     -0.044103
2012-12-31 00:00:00-05:00      0.164367
2013-12-31 00:00:00-05:00      0.259992
2014-12-31 00:00:00-05:00      0.118761
2015-12-31 00:00:00-05:00     -0.037058
2016-12-31 00:00:00-05:00      0.233812
2017-12-31 00:00:00-05:00      0.125416
2018-12-31 00:00:00-05:00     -0.097634
2019-12-31 00:00:00-05:00      0.206600
2020-12-31 00:00:00-05:00      0.009886
2021-12-31 00:00:00-05:00      0.355708
2022-12-31 00:00:00-05:00     -0.099170
2023-12

# Example 2 EndOfYear- AdjPrice, dividendSum, the yearly total return
from code interpreter

In [ ]:
import yfinance as yf

def get_yearly_data(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)

    history = stock.history(period="max").reset_index()
    # Extract the year from the Date column
    history['Year'] = history['Date'].dt.year

    # Group the data by year and get the last trading date, Adj Close, and sum of dividends for each year
    yearly_data = history.groupby('Year').agg({'Date': 'last', 'Close': 'last', 'Dividends': 'sum'})
    yearly_data.rename(columns={'Date': 'LastTradingDate', 'Close': 'Adj Close', 'Dividends': 'DividendSum'}, inplace=True)

    # Calculate the total return for each year
    yearly_data['TotalReturn'] = yearly_data['Adj Close'] / yearly_data['Adj Close'].shift(1) - 1
    yearly_data['TotalReturn'] = yearly_data['TotalReturn'] * 100
    yearly_data['TotalReturn'] = yearly_data['TotalReturn'].map("{:.2f}%".format)

    # Format the "Last Trading Date" column to display only the date portion
    yearly_data['LastTradingDate'] = yearly_data['LastTradingDate'].dt.date
    return yearly_data

# Main program
ticker = 'XFN.TO'
data = get_yearly_data(ticker)

print(data)



     LastTradingDate  Adj Close  DividendSum TotalReturn
Year                                                    
2001      2001-12-31   7.318615       0.1875        nan%
2002      2002-12-31   7.094312       0.2825      -3.06%
2003      2003-12-31   8.948480       0.3455      26.14%
2004      2004-12-31  10.610480       0.3805      18.57%
2005      2005-12-30  13.040280       0.4635      22.90%
2006      2006-12-29  15.360328       0.5420      17.79%
2007      2007-12-31  15.138770       0.6590      -1.44%
2008      2008-12-31   9.691955       0.8115     -35.98%
2009      2009-12-31  14.012818       0.8150      44.58%
2010      2010-12-31  15.120352       0.7440       7.90%
2011      2011-12-30  14.453518       0.7970      -4.41%
2012      2012-12-31  16.829203       0.8220      16.44%
2013      2013-12-31  21.204639       0.8320      26.00%
2014      2014-12-31  23.722937       0.8830      11.88%
2015      2015-12-31  22.843807       0.9390      -3.71%
2016      2016-12-30  28.184969

# Example 3: retrieve  current last stock price from yahoo finance


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime

def get_current_prices(tickers_list):
    # Create an empty dictionary to store data
    data = {}
    # Get current prices for each ticker
    for ticker in tickers_list:
        # Create a ticker object
        stock = yf.Ticker(ticker)
        # Get the current market last price
        last_price = stock.basic_info['last_price']
        # Store the price in the dictionary
        data[ticker] = [last_price]
    # Create a DataFrame from the dictionary
    df = pd.DataFrame(data)
    return df

# Define the list of tickers
tickers_list = ["spy", "vfv.to", "xiu.to", "xic.to", "xfn.to", "ry.to", "enb.to"]
# Call the function to get the DataFrame
current_prices_df = get_current_prices(tickers_list)

current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Market last price from Yahoo Finance (Current time: {current_datetime})")
# Print the returned DataFrame with only 2 digit after dot, without index
print(current_prices_df.round(2).to_string(index=False))



Market last price from Yahoo Finance (Current time: 2023-12-21 02:41:31)
   spy  vfv.to  xiu.to  xic.to  xfn.to  ry.to  enb.to
468.26  111.66   31.42   32.93   47.15 132.33   47.28


# a simple example

In [ ]:
#!pip install yfinance
import yfinance as yf
ticker = "XFN.TO"
# Create a ticker object
stock = yf.Ticker(ticker)
# Get the current market price
price = stock.basic_info['last_price']
print(f"The last_price price of {ticker} is ${price:.4f}")



The last_price price of XFN.TO is $46.9000


#  backup-Example 2 -endofyear price, dividend sum, the yearly total return;  from code interpreter

Would you adjust the python program according to this requirement (percentage format is to be used for “the total return of the year", with % character): ### Giving a stock ticker (for example XFN.to), the python program takes the data from yahoo finance, and list "last trading date of the year", “prices of Adj Close** at the last trading date of the year”, “ sum of dividends during the year” (by adding all distributed dividends during the year), “the total return of the year (with % character)”, for all years with available data. the date portion displays year/month/date part only.
Please use the stocker XFN.to in the program ###

In [ ]:
import yfinance as yf

def get_yearly_data(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)

    history = stock.history(period="max").reset_index()
    # Extract the year from the Date column
    history['Year'] = history['Date'].dt.year

    # Group the data by year and get the last trading date, Adj Close, and sum of dividends for each year
    yearly_data = history.groupby('Year').agg({'Date': 'last', 'Close': 'last', 'Dividends': 'sum'})
    yearly_data.rename(columns={'Date': 'LastTradingDate', 'Close': 'Adj Close', 'Dividends': 'DividendSum'}, inplace=True)

    # Calculate the total return for each year
    yearly_data['TotalReturn'] = yearly_data['Adj Close'] / yearly_data['Adj Close'].shift(1) - 1
    yearly_data['TotalReturn'] = yearly_data['TotalReturn'] * 100
    yearly_data['TotalReturn'] = yearly_data['TotalReturn'].map("{:.2f}%".format)

    # Format the "Last Trading Date" column to display only the date portion
    yearly_data['LastTradingDate'] = yearly_data['LastTradingDate'].dt.date
    return yearly_data

# Main program
ticker = 'XFN.TO'
data = get_yearly_data(ticker)

print(data)



     LastTradingDate  Adj Close  DividendSum TotalReturn
Year                                                    
2001      2001-12-31   7.318617       0.1875        nan%
2002      2002-12-31   7.094311       0.2825      -3.06%
2003      2003-12-31   8.948478       0.3455      26.14%
2004      2004-12-31  10.610490       0.3805      18.57%
2005      2005-12-30  13.040289       0.4635      22.90%
2006      2006-12-29  15.360311       0.5420      17.79%
2007      2007-12-31  15.138773       0.6590      -1.44%
2008      2008-12-31   9.691961       0.8115     -35.98%
2009      2009-12-31  14.012826       0.8150      44.58%
2010      2010-12-31  15.120366       0.7440       7.90%
2011      2011-12-30  14.453509       0.7970      -4.41%
2012      2012-12-31  16.829199       0.8220      16.44%
2013      2013-12-31  21.204638       0.8320      26.00%
2014      2014-12-31  23.722931       0.8830      11.88%
2015      2015-12-31  22.843803       0.9390      -3.71%
2016      2016-12-30  28.184971

# trailing total return from hard-coded data input


Certainly! To convert the provided data into a 2-dimensional array in Python, you can use a list of lists. Here's how you can represent the data:

python
Copy code
data_array = [
    ["Year", "Total Return %"],
    [2013, 26.00],
    [2014, 11.89],
    [2015, -3.71],
    [2016, 23.36],
    [2017, 12.54],
    [2018, -9.76],
    [2019, 20.66],
    [2020, 0.98],
    [2021, 35.56],
    [2022, -9.93],
    ["YTD", None]  # Assuming YTD is a placeholder, set to None or provide the actual value
]

In [ ]:
import yfinance as yf

ticker = 'XFN.TO'
# data from Ishare web site
# xfn_yearly_total_return as input
xfn_yearly_total_return = [
    ['Year', 'Total Return'],
    [2013, 26.0],
    [2014, 11.89],
    [2015, -3.71],
    [2016, 23.36],
    [2017, 12.54],
    [2018, -9.76],
    [2019, 20.66],
    [2020, 0.98],
    [2021, 35.56],
    [2022, -9.93],
]
# expect output according Ishare web
xfn_trailing_return_At_2022_END = [
    ["years", "1y", "3y", "5y", "10y"],
    ["Total Return %", -9.90, 7.24, 6.07, 9.73]
]

# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(data, years):
    # Get the total return values for the last n years
    trailing_data = data['Total Return'].tail(years)

    # Convert the percentage strings to numeric values
    #trailing_data = trailing_data.str.replace('%', '').astype(float)
    trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)

    # Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
    # Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
    # Where r1, r2, ..., rn are the total return values for each year
    annualized_return = (trailing_data / 100 + 1).prod() ** (1 / years) - 1

    # Format the result as a percentage with two decimal places
    annualized_return = "{:.2f}%".format(annualized_return * 100)
    return annualized_return

TRAIL_RETURN_AS_YEAR=2022

# Define a list of years to calculate the trailing returns
years_list = [1, 3, 5, 10]

# please add a code section to convert variable 'xfn_yearly_total_return'
# to variable 'data' for get_trailing_return(data, years)
# Convert the list of lists to a pandas dataframe
import pandas as pd
data = pd.DataFrame(xfn_yearly_total_return[1:], columns=xfn_yearly_total_return[0])
# Set the index to the year column
data.set_index('Year', inplace=True)
print(data)

# Loop through the list and print the trailing returns for each year
for years in years_list:
    trailing_return = get_trailing_return(data, years)
    print(f"{years}-Year {trailing_return}")


      Total Return
Year              
2013         26.00
2014         11.89
2015         -3.71
2016         23.36
2017         12.54
2018         -9.76
2019         20.66
2020          0.98
2021         35.56
2022         -9.93
1-Year -9.93%
3-Year 7.23%
5-Year 6.07%
10-Year 9.73%


# An example for cumulative turn -  calculate the trailing total return (dividend not considered)

In [ ]:
# Import libraries
import yfinance as yf
import pandas as pd

# Define the stock ticker and the time periods
ticker = "XFN.to"
periods = [1, 3, 5, 10] # in years

# Fetch the historical data
stock = yf.Ticker(ticker)
data = stock.history(period="max")

# Calculate the annualized trailing return for each period
for period in periods:
    # Get the close prices for the period
    close = data["Close"].tail(period * 252) # 252 trading days in a year

    # Calculate the percentage change
    pct_change = close.pct_change()

    # Calculate the annualized trailing return
    atr = (1 + pct_change).prod() ** (1 / period) - 1

    # Print the result
    print(f"The annualized trailing return for {ticker} over {period} year(s) is {atr:.2%}")



The annualized trailing return for XFN.to over 1 year(s) is 9.39%
The annualized trailing return for XFN.to over 3 year(s) is 9.95%
The annualized trailing return for XFN.to over 5 year(s) is 9.67%
The annualized trailing return for XFN.to over 10 year(s) is 8.32%


# bachup - Example 1: Retrieve multiple tickers for total returns and trailing total returns

In [17]:
"""
### CODE start (for AI tool)
Retrieve multiple tickers for total returns and trailing total returns from yahoo
History:
Dec 21, 2023, version 12

Author: Gang Luo
"""
result_marker='###'
script_version = 12
print(f"The result of version-{script_version} script  start:\n {result_marker}")
#!pip install yfinance
import yfinance as yf
import pandas as pd

# Set the stock tickers list
#tickers_list = ["spy", "vfv.to", "xiu.to", "xic.to", "xfn.to", "ry.to", "enb.to"]
tickers_list = ["xiu.to", "xfn.to", "ry.to", "td.to", "na.to", "cwb.to", "slf.to", "gwo.to"]
#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display

# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    ''' Get annual total return data.   Explanation the following statement
    1.  'Close'  in .Ticker(ticker).history is the same as 'Adj Close' in .download(ticker, ...)
        print(df.columns.tolist()) - Display all column names of DataFrame
        yf.Ticker(ticker).history: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
        yf.download(ticker, ...): ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    2.  ["Close"]: Selects the "Close" column from the historical stock data.
        .resample('Y'): Resamples the time series data annually ('Y').
        .ffill(): Forward fills missing values, ensuring that each annual period has a value.
        .pct_change(): Calculates the percentage change between the current and previous values, representing the annual returns.
        .dropna(): Removes any rows with missing values (which may occur after calculating percentage changes).
    '''
    annual_returns = stock.history(period="max")["Close"].resample('Y').ffill().pct_change().dropna()
    annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
    annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
    return annual_returns_df
# Create an empty DataFrame to store all tickers' total returns
all_tickers_returns_df = pd.DataFrame()

# Loop through each ticker in the list
for ticker in tickers_list:
    ticker_returns_df = get_annual_returns_df(ticker)
    if not ticker_returns_df.empty:
        if all_tickers_returns_df.empty:
            all_tickers_returns_df = ticker_returns_df
        else:
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df], axis=1)  # Concatenate DataFrames

# Display the results if the DataFrame is not empty
print("\nAnnual Total Return (%) History:")
if not all_tickers_returns_df.empty:
    # keep the date portion of index only - stripping time portion
    #all_tickers_returns_df.index=all_tickers_returns_df.index.date

    # Format the DataFrame to display percentage
    all_tickers_returns_df_copy = all_tickers_returns_df * 100
    all_tickers_returns_df_copy=all_tickers_returns_df_copy.round(2)
    all_tickers_returns_df_copy.index=all_tickers_returns_df_copy.index.date
    print(all_tickers_returns_df_copy)
else:
    print("No data available for the specified tickers.")

#-------------------------------------------------------------------
# step 2: Remove the row of the current year for the later steps
# Define a list of years to calculate the trailing returns, cumulative returns, and so on
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30]
current_year=all_tickers_returns_df.index.year.max()
all_tickers_returns_df['Year']=all_tickers_returns_df.index.year
all_tickers_returns_df=all_tickers_returns_df[all_tickers_returns_df['Year'] < current_year]


#-------------------------------------------------------------------
# step 3: calculate the annualized trailing total return from the data generated in step 1 & display
# Define a function to calculate the annualized trailing total return for a given number of years
def get_trailing_return(ticker, data, years):
    # Get the total return values for the last n years
    trailing_data = data[ticker].tail(years)

    # Check if there are valid total return values for all years
    if len(trailing_data) == years:
        # Convert the percentage strings to numeric values
        trailing_data = trailing_data.astype(str).str.replace('%', '').astype(float)

        """ Calculate the annualized trailing total return using the formula from Investopedia[^1^][1]:
            Annualized Return = [(1 + r1) * (1 + r2) * ... * (1 + rn)]^(1/n) - 1
            Where r1, r2, ..., rn are the total return values for each year                    """
        annualized_trailing_return = (trailing_data + 1).prod() ** (1 / years) - 1

        # Format the result as a percentage with two decimal places
        annualized_trailing_return = annualized_trailing_return * 100
        annualized_trailing_return = annualized_trailing_return.round(2)
        return annualized_trailing_return
    else:
        return "N/A"

# Create an empty DataFrame with years_list as the index for trailing total returns
all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

# Define a function to Loop through the list and print the trailing returns for each num_years
def get_trailing_return_column(ticker):
    trailing_return_column = {}
    for num_years in years_list:
        # Check if the ticker data is available in all_tickers_returns_df
        if ticker in all_tickers_returns_df.columns:
            # using data from step 1, avoiding get_annual_returns_df(ticker) for less traffic from yahoo server
            data = all_tickers_returns_df[[ticker]]
            trailing_return = get_trailing_return(ticker, data, num_years)
            trailing_return_column[f"{num_years}-Year"] = trailing_return
        else:
            print(f"Data not available for {ticker}. Skipping.")
            trailing_return_column[f"{num_years}-Year"] = "N/A"
    return trailing_return_column

# Create an empty DataFrame to store all tickers' trailing returns
all_tickers_trailing_returns_df = pd.DataFrame(index=years_list)

# Loop through each ticker in the list
for ticker in tickers_list:
    trailing_returns = get_trailing_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_trailing_returns_df[ticker] = pd.Series(trailing_returns).values

# Display the trailing returns DataFrame
print("\nAnnualized Trailing Returns as end of year", current_year-1, ":")
print("years")
print(all_tickers_trailing_returns_df)

# print an indicator to mark the end of execution result
print(f"The result of version-{script_version} script  ends here.\n {result_marker}")
### CODE end (for AI tool)


The result of version-12 script  start:
 ###

Annual Total Return (%) History:
            xiu.to  xfn.to  ry.to  td.to   na.to  cwb.to  slf.to  gwo.to
1996-12-31     NaN     NaN  59.20  52.30   29.73   38.08     NaN     NaN
1997-12-31     NaN     NaN  61.23  56.87   75.97   62.90     NaN     NaN
1998-12-31     NaN     NaN   3.58   2.50    7.92   10.75     NaN     NaN
1999-12-31     NaN     NaN -14.82  46.56  -22.65  -21.16     NaN     NaN
2000-12-31    8.02     NaN  65.34  14.84   43.78   42.46     NaN     NaN
2001-12-31  -14.24     NaN   4.90  -2.81   11.65    9.86  -13.63     NaN
2002-12-31  -14.02   -3.06  14.85 -14.56    8.75   -5.13  -19.87     NaN
2003-12-31   24.82   26.14   9.13  31.37   35.63   50.03   23.78   25.64
2004-12-31   13.28   18.57   7.38  18.83   18.76   37.62   27.27   20.62
2005-12-31   25.47   22.90  45.87  26.19   25.57   36.52   19.14   18.24
2006-12-31   19.12   17.79  26.02  16.55   12.71   49.19    8.15   13.56
2007-12-31   10.83   -1.44  -5.57   2.66  -17

# Example 4:  Cumulative Returns & Compound Annual Growth Rate

In [19]:
"""
### CODE start (for AI tool)
Retrieve multiple tickers for total returns and trailing total returns from yahoo
History:
Dec 21, version 1,  2023

Author: Gang Luo
"""
result_marker='###'
script_version = 1
print(f"The result of version-{script_version} script  start:\n {result_marker}")
#!pip install yfinance
import yfinance as yf
import pandas as pd

# Set the stock tickers list
#tickers_list = ["spy", "vfv.to", "xiu.to", "xic.to", "xfn.to", "ry.to", "enb.to"]
tickers_list = ["xiu.to", "xfn.to", "ry.to", "td.to", "na.to", "cwb.to", "slf.to", "gwo.to"]
#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display

# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    ''' Get annual total return data.   Explanation the following statement
    1.  'Close'  in .Ticker(ticker).history is the same as 'Adj Close' in .download(ticker, ...)
        print(df.columns.tolist()) - Display all column names of DataFrame
        yf.Ticker(ticker).history: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
        yf.download(ticker, ...): ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    2.  ["Close"]: Selects the "Close" column from the historical stock data.
        .resample('Y'): Resamples the time series data annually ('Y').
        .ffill(): Forward fills missing values, ensuring that each annual period has a value.
        .pct_change(): Calculates the percentage change between the current and previous values, representing the annual returns.
        .dropna(): Removes any rows with missing values (which may occur after calculating percentage changes).
    '''
    annual_returns = stock.history(period="max")["Close"].resample('Y').ffill().pct_change().dropna()
    annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
    annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
    return annual_returns_df
# Create an empty DataFrame to store all tickers' total returns
all_tickers_returns_df = pd.DataFrame()

# Loop through each ticker in the list
for ticker in tickers_list:
    ticker_returns_df = get_annual_returns_df(ticker)
    if not ticker_returns_df.empty:
        if all_tickers_returns_df.empty:
            all_tickers_returns_df = ticker_returns_df
        else:
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df], axis=1)  # Concatenate DataFrames

# Display the results if the DataFrame is not empty
print("\nAnnual Total Return (%) History:")
if not all_tickers_returns_df.empty:
    # keep the date portion of index only - stripping time portion
    #all_tickers_returns_df.index=all_tickers_returns_df.index.date

    # Format the DataFrame to display percentage
    all_tickers_returns_df_copy = all_tickers_returns_df * 100
    all_tickers_returns_df_copy=all_tickers_returns_df_copy.round(2)
    all_tickers_returns_df_copy.index=all_tickers_returns_df_copy.index.date
    print(all_tickers_returns_df_copy)
else:
    print("No data available for the specified tickers.")

#-------------------------------------------------------------------
# step 2: Remove the row of the current year for the later steps
# Define a list of years to calculate the trailing returns, cumulative returns, and so on
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30]
current_year=all_tickers_returns_df.index.year.max()
all_tickers_returns_df['Year']=all_tickers_returns_df.index.year
all_tickers_returns_df=all_tickers_returns_df[all_tickers_returns_df['Year'] < current_year]

#-------------------------------------------------------------------
# step 4: calculate the cumulative return from the data (all_tickers_returns_df) generated in step 1 & display
# 4.1 Define a function to calculate the cumulative return for a given number of years from a ticker
def get_cumulative_return(ticker, data, years):
    # Calculate the cumulative return
    cumulative_return = (1 + data[ticker]).rolling(window=years).apply(lambda x: x.prod(), raw=True) - 1
    return cumulative_return

# Create an empty DataFrame with years_list as the index for cumulative  returns
all_tickers_cumulative_returns_df = pd.DataFrame(index=years_list)

# Define a function to Loop through the list and return the cumulative returns for each num_years
def get_cumulative_return_column(ticker):
    cumulative_returns = {}
    for years in years_list:
        # Calculate the cumulative return for the given number of years
        cumulative_return = get_cumulative_return(ticker, all_tickers_returns_df, years)
        # Get the last value, which is the cumulative return up to the current year
        cumulative_returns[years] = cumulative_return.iloc[-1]
    return cumulative_returns

# Loop through each ticker in the list
for ticker in tickers_list:
    cumulative_returns = get_cumulative_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_cumulative_returns_df[ticker] = pd.Series(cumulative_returns).values

# Display the cumulative returns DataFrame
print("\nCumulative Returns as end of year", current_year-1, ":")
print("years")
print(all_tickers_cumulative_returns_df.round(2) * 100)

#-------------------------------------------------------------------
# step 5: calculate the  CAGR from the data (all_tickers_cumulative_returns_df) generated earlier & display
# Define a function to calculate the CAGR from the cumulative value and the years
import numpy as np
def calculate_cagr(value, years):
    #print("debug-value\n", value, "end")
    #print("debug-years\n", years, "end")
    # Otherwise, calculate the CAGR using the formula
    cagr = (value + 1) ** (1 / np.array(years)) - 1
    #print("debug-cagr\n", cagr, "end")
    return cagr

'''
def calculate_cagr(value, years):
    # If the value is NaN, return NaN
    if pd.isna(value):
        return np.nan
    # Otherwise, calculate the CAGR using the formula
    else:
        cagr = (value + 1) ** (1 / years) - 1
        return cagr
'''

# Define a function to format the Float64Index values into percentage strings
def format_to_percentage(value):
    # If any element in the value array is not null, format it as a percentage string with two decimal places
    if np.any(pd.notnull(value)):
        return f"{value:.2f}%"
    # Otherwise, return None
    return None

# Apply the formatting function to each column of all_tickers_cagrs_df
#all_tickers_cagrs_df = all_tickers_cagrs_df.applymap(format_to_percentage)

# Apply the calculate_cagr function to each column of the DataFrame
all_tickers_cagrs_df = all_tickers_cumulative_returns_df.apply(lambda x: calculate_cagr(x, x.index), axis=0)
# Apply the calculate_cagr function to each element of the DataFrame
#all_tickers_cagrs_df = all_tickers_cumulative_returns_df.applymap(lambda x: calculate_cagr(x, all_tickers_cumulative_returns_df.index))
#all_tickers_cagrs_df = all_tickers_cagrs_df.applymap(format_to_percentage)

# Display the formatted DataFrame
print("\nCAGRs (%) as end of year", current_year-1, ":")
print("years")
print(all_tickers_cagrs_df.round(2) * 100)


The result of version-1 script  start:
 ###

Annual Total Return (%) History:
            xiu.to  xfn.to  ry.to  td.to   na.to  cwb.to  slf.to  gwo.to
1996-12-31     NaN     NaN  59.20  52.30   29.73   38.08     NaN     NaN
1997-12-31     NaN     NaN  61.23  56.87   75.97   62.90     NaN     NaN
1998-12-31     NaN     NaN   3.58   2.50    7.92   10.75     NaN     NaN
1999-12-31     NaN     NaN -14.82  46.56  -22.65  -21.16     NaN     NaN
2000-12-31    8.02     NaN  65.34  14.84   43.78   42.46     NaN     NaN
2001-12-31  -14.24     NaN   4.90  -2.81   11.65    9.86  -13.63     NaN
2002-12-31  -14.02   -3.06  14.85 -14.56    8.75   -5.13  -19.87     NaN
2003-12-31   24.82   26.14   9.13  31.37   35.63   50.03   23.78   25.64
2004-12-31   13.28   18.57   7.38  18.83   18.76   37.62   27.27   20.62
2005-12-31   25.47   22.90  45.87  26.19   25.57   36.52   19.14   18.24
2006-12-31   19.12   17.79  26.02  16.55   12.71   49.19    8.15   13.56
2007-12-31   10.83   -1.44  -5.57   2.66  -17.

# back-up example 4

In [20]:
"""
### CODE start (for AI tool)
Retrieve multiple tickers for total returns and trailing total returns from yahoo
History:
Dec 21, version 1,  2023

Author: Gang Luo
"""
result_marker='###'
script_version = 1
print(f"The result of version-{script_version} script  start:\n {result_marker}")
#!pip install yfinance
import yfinance as yf
import pandas as pd

# Set the stock tickers list
#tickers_list = ["spy", "vfv.to", "xiu.to", "xic.to", "xfn.to", "ry.to", "enb.to"]
tickers_list = ["xiu.to", "xfn.to", "ry.to", "td.to", "na.to", "cwb.to", "slf.to", "gwo.to"]
#-------------------------------------------------------------------
# step 1: fetch retrieve yearly total returns by yfinance & display

# Function to fetch data from yfinance and extract yearly total returns
def get_annual_returns_df(ticker):
    # Get the historical data for the given ticker
    stock = yf.Ticker(ticker)
    ''' Get annual total return data.   Explanation the following statement
    1.  'Close'  in .Ticker(ticker).history is the same as 'Adj Close' in .download(ticker, ...)
        print(df.columns.tolist()) - Display all column names of DataFrame
        yf.Ticker(ticker).history: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
        yf.download(ticker, ...): ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    2.  ["Close"]: Selects the "Close" column from the historical stock data.
        .resample('Y'): Resamples the time series data annually ('Y').
        .ffill(): Forward fills missing values, ensuring that each annual period has a value.
        .pct_change(): Calculates the percentage change between the current and previous values, representing the annual returns.
        .dropna(): Removes any rows with missing values (which may occur after calculating percentage changes).
    '''
    annual_returns = stock.history(period="max")["Close"].resample('Y').ffill().pct_change().dropna()
    annual_returns_df = pd.DataFrame(annual_returns, columns=['Close'])
    annual_returns_df.rename(columns={'Close': ticker}, inplace=True)
    return annual_returns_df
# Create an empty DataFrame to store all tickers' total returns
all_tickers_returns_df = pd.DataFrame()

# Loop through each ticker in the list
for ticker in tickers_list:
    ticker_returns_df = get_annual_returns_df(ticker)
    if not ticker_returns_df.empty:
        if all_tickers_returns_df.empty:
            all_tickers_returns_df = ticker_returns_df
        else:
            all_tickers_returns_df = pd.concat([all_tickers_returns_df, ticker_returns_df], axis=1)  # Concatenate DataFrames

# Display the results if the DataFrame is not empty
print("\nAnnual Total Return (%) History:")
if not all_tickers_returns_df.empty:
    # keep the date portion of index only - stripping time portion
    #all_tickers_returns_df.index=all_tickers_returns_df.index.date

    # Format the DataFrame to display percentage
    all_tickers_returns_df_copy = all_tickers_returns_df * 100
    all_tickers_returns_df_copy=all_tickers_returns_df_copy.round(2)
    all_tickers_returns_df_copy.index=all_tickers_returns_df_copy.index.date
    print(all_tickers_returns_df_copy)
else:
    print("No data available for the specified tickers.")

#-------------------------------------------------------------------
# step 2: Remove the row of the current year for the later steps
# Define a list of years to calculate the trailing returns, cumulative returns, and so on
years_list = [1, 2, 3, 5, 10, 15, 20, 25, 30]
current_year=all_tickers_returns_df.index.year.max()
all_tickers_returns_df['Year']=all_tickers_returns_df.index.year
all_tickers_returns_df=all_tickers_returns_df[all_tickers_returns_df['Year'] < current_year]

#-------------------------------------------------------------------
# step 4: calculate the cumulative return from the data (all_tickers_returns_df) generated in step 1 & display
# 4.1 Define a function to calculate the cumulative return for a given number of years from a ticker
def get_cumulative_return(ticker, data, years):
    # Calculate the cumulative return
    cumulative_return = (1 + data[ticker]).rolling(window=years).apply(lambda x: x.prod(), raw=True) - 1
    return cumulative_return

# Create an empty DataFrame with years_list as the index for cumulative  returns
all_tickers_cumulative_returns_df = pd.DataFrame(index=years_list)

# Define a function to Loop through the list and return the cumulative returns for each num_years
def get_cumulative_return_column(ticker):
    cumulative_returns = {}
    for years in years_list:
        # Calculate the cumulative return for the given number of years
        cumulative_return = get_cumulative_return(ticker, all_tickers_returns_df, years)
        # Get the last value, which is the cumulative return up to the current year
        cumulative_returns[years] = cumulative_return.iloc[-1]
    return cumulative_returns

# Loop through each ticker in the list
for ticker in tickers_list:
    cumulative_returns = get_cumulative_return_column(ticker)
    # Add the trailing returns to the DataFrame
    all_tickers_cumulative_returns_df[ticker] = pd.Series(cumulative_returns).values

# Display the cumulative returns DataFrame
print("\nCumulative Returns as end of year", current_year-1, ":")
print("years")
print(all_tickers_cumulative_returns_df.round(2) * 100)

#-------------------------------------------------------------------
# step 5: calculate the  CAGR from the data (all_tickers_cumulative_returns_df) generated earlier & display
# Define a function to calculate the CAGR from the cumulative value and the years
import numpy as np
def calculate_cagr(value, years):
    #print("debug-value\n", value, "end")
    #print("debug-years\n", years, "end")
    # Otherwise, calculate the CAGR using the formula
    cagr = (value + 1) ** (1 / np.array(years)) - 1
    #print("debug-cagr\n", cagr, "end")
    return cagr

'''
def calculate_cagr(value, years):
    # If the value is NaN, return NaN
    if pd.isna(value):
        return np.nan
    # Otherwise, calculate the CAGR using the formula
    else:
        cagr = (value + 1) ** (1 / years) - 1
        return cagr
'''

# Define a function to format the Float64Index values into percentage strings
def format_to_percentage(value):
    # If any element in the value array is not null, format it as a percentage string with two decimal places
    if np.any(pd.notnull(value)):
        return f"{value:.2f}%"
    # Otherwise, return None
    return None

# Apply the formatting function to each column of all_tickers_cagrs_df
#all_tickers_cagrs_df = all_tickers_cagrs_df.applymap(format_to_percentage)

# Apply the calculate_cagr function to each column of the DataFrame
all_tickers_cagrs_df = all_tickers_cumulative_returns_df.apply(lambda x: calculate_cagr(x, x.index), axis=0)
# Apply the calculate_cagr function to each element of the DataFrame
#all_tickers_cagrs_df = all_tickers_cumulative_returns_df.applymap(lambda x: calculate_cagr(x, all_tickers_cumulative_returns_df.index))
#all_tickers_cagrs_df = all_tickers_cagrs_df.applymap(format_to_percentage)

# Display the formatted DataFrame
print("\nCAGRs (%) as end of year", current_year-1, ":")
print("years")
print(all_tickers_cagrs_df.round(2) * 100)


The result of version-1 script  start:
 ###

Annual Total Return (%) History:
            xiu.to  xfn.to  ry.to  td.to   na.to  cwb.to  slf.to  gwo.to
1996-12-31     NaN     NaN  59.20  52.30   29.73   38.08     NaN     NaN
1997-12-31     NaN     NaN  61.23  56.87   75.97   62.90     NaN     NaN
1998-12-31     NaN     NaN   3.58   2.50    7.92   10.75     NaN     NaN
1999-12-31     NaN     NaN -14.82  46.56  -22.65  -21.16     NaN     NaN
2000-12-31    8.02     NaN  65.34  14.84   43.78   42.47     NaN     NaN
2001-12-31  -14.24     NaN   4.90  -2.81   11.65    9.86  -13.63     NaN
2002-12-31  -14.02   -3.06  14.85 -14.56    8.75   -5.13  -19.87     NaN
2003-12-31   24.82   26.14   9.13  31.37   35.63   50.03   23.78   25.64
2004-12-31   13.28   18.57   7.38  18.83   18.76   37.62   27.27   20.62
2005-12-31   25.47   22.90  45.87  26.19   25.57   36.52   19.14   18.24
2006-12-31   19.12   17.79  26.02  16.55   12.71   49.19    8.15   13.56
2007-12-31   10.83   -1.44  -5.57   2.66  -17.